In [48]:
import numpy as np
import pandas as pd
import random
from collections import Counter

In [15]:
class LoadPreprocessData:
    def __init__(self, path, header = None):
        self.dataset = pd.read_csv(path, header = header)
        self.X = self.dataset.iloc[:, :4]
        self.y = self.dataset.iloc[:, 4]
        
    def normalize(self, axis = 0):
        return ((self.X - self.X.mean(axis = axis)) / self.X.std(axis = axis))
    
    def train_test_split(self, percent = 80):
        train_rows = random.sample(range(0, self.y.size), percent * self.y.size // 100)
        train_rows.sort()
        test_rows=[rows for rows in self.X.index.values if rows not in train_rows]
        self.train_X = self.X.iloc[train_rows].reset_index(drop = True) 
        self.train_y = self.y.iloc[train_rows].reset_index(drop = True) 
        
        self.test_X = self.X.iloc[test_rows].reset_index(drop = True) 
        self.test_y = self.y.iloc[test_rows].reset_index(drop = True) 

In [16]:
dataClass = LoadPreprocessData("data_banknote_authentication.txt", None)
dataClass.normalize()
dataClass.train_test_split()
dataClass.train_X

,0,1,2,3
0,3.62160,8.66610,-2.80730,-0.44699
1,4.54590,8.16740,-2.45860,-1.46210
2,3.86600,-2.63830,1.92420,0.10645
3,3.45660,9.52280,-4.01120,-3.59440
4,0.32924,-4.45520,4.57180,-0.98880
...,...,...,...,...
1092,-2.41000,3.74330,-0.40215,-1.29530
1093,0.40614,1.34920,-1.45010,-0.55949
1094,-3.75030,-13.45860,17.59320,-2.77710
1095,-3.56370,-8.38270,12.39300,-1.28230


In [17]:
class LogisticRegression:
    def __init__(self, lr=0.01, reg_factor = 0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.reg_factor = reg_factor
    def initialize_weights(self, X, weights = "0"):
        if weights == "0":
            self.theta = np.zeros(X.shape[1])
        elif weights == "gaussian":
            self.theta = np.random.randn(X.shape[1])
        elif weights == "uniform":
            self.theta = np.random.uniform(size = X.shape[1])
        elif weights == "xavier":
            self.theta = np.random.randn(X.shape[1]) * np.sqrt(1 / X.shape[1])
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y, type = "no regularisation"):
        if(type == "no regularisation"):
            return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        
        elif type == "L1":
            return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean() - 0.5 * self.reg_factor * np.sum(np.abs(self.theta)) / y.size
        
        else:
            return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean() - 0.5 * self.reg_factor * np.sum(self.theta ** 2) / y.size
    
    def fit(self, X, y , type = "no regularisation", weights = "0"):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        self.initialize_weights(X, weights)
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            if type == "no regularisation":
                gradient = np.dot(X.T, (h - y))
                
            elif type == "L1":
                gradient = np.dot(X.T, (h - y)) + 0.5 * self.reg_factor * np.sign(self.theta)
                
            else:
                gradient = np.dot(X.T, (h - y)) + 0.5 * self.reg_factor * self.theta 
                
            self.theta -= self.lr * gradient / y.size
            
            if(self.verbose == True and i % 5000 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y, type)} Iteration: {i}\t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

In [235]:
model = LogisticRegression(lr=0.1, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "uniform")

loss: 1.9745345089337842 Iteration: 0	
loss: 0.022386822465845134 Iteration: 5000	
loss: 0.020080813339752132 Iteration: 10000	
loss: 0.01912433048377967 Iteration: 15000	
Wall time: 5.06 s


In [236]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9818181818181818

In [237]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [238]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [239]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9818181818181818

In [240]:
accuracy.F_score()
accuracy.fScore

0.979757085020243

In [147]:
model = LogisticRegression(lr=0.1, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "xavier")

loss: 0.8049980233446538 Iteration: 0	
loss: 0.022437208221555795 Iteration: 5000	
loss: 0.02009725433724085 Iteration: 10000	
loss: 0.019132763613749233 Iteration: 15000	
Wall time: 5 s


In [148]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9818181818181818

In [149]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [150]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [151]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9818181818181818

In [152]:
accuracy.F_score()
accuracy.fScore

0.979757085020243

In [153]:
model = LogisticRegression(lr=0.1, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "0")

loss: 0.5102929335389772 Iteration: 0	
loss: 0.02245830276133439 Iteration: 5000	
loss: 0.02010408943527672 Iteration: 10000	
loss: 0.019136261871582672 Iteration: 15000	
Wall time: 5.02 s


In [154]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9818181818181818

In [155]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [156]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [157]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9818181818181818

In [158]:
accuracy.F_score()
accuracy.fScore

0.979757085020243

In [159]:
model = LogisticRegression(lr=0.1, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "gaussian")

loss: 2.0639457138876307 Iteration: 0	
loss: 0.022372157732030114 Iteration: 5000	
loss: 0.020075989383641243 Iteration: 10000	
loss: 0.019121851119578898 Iteration: 15000	
Wall time: 5.09 s


In [160]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9818181818181818

In [161]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [162]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [163]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9818181818181818

In [164]:
accuracy.F_score()
accuracy.fScore

0.979757085020243

In [165]:
model = LogisticRegression(lr=0.01, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "0")

loss: 0.6631208142724372 Iteration: 0	
loss: 0.046819128936556036 Iteration: 5000	
loss: 0.03487283920471139 Iteration: 10000	
loss: 0.030261329539015534 Iteration: 15000	
Wall time: 5.03 s


In [166]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9854545454545455

In [167]:
weights = model.theta
weights

array([ 2.72069982, -2.56213172, -1.54552971, -1.81023256, -0.17765387])

In [168]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [169]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [170]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9854545454545455

In [171]:
accuracy.F_score()
accuracy.fScore

0.9838709677419355

In [172]:
model = LogisticRegression(lr=0.01, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "uniform")

loss: 2.002107878247501 Iteration: 0	
loss: 0.04530475711981196 Iteration: 5000	
loss: 0.03442915685179356 Iteration: 10000	
loss: 0.030057404796394824 Iteration: 15000	
Wall time: 5 s


In [173]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9854545454545455

In [174]:
weights = model.theta
weights

array([ 2.73874193, -2.56977832, -1.54869569, -1.81534201, -0.17389282])

In [175]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [176]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [177]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9854545454545455

In [178]:
accuracy.F_score()
accuracy.fScore

0.9838709677419355

In [179]:
model = LogisticRegression(lr=0.01, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "xavier")

loss: 3.4514797226080627 Iteration: 0	
loss: 0.0481342492808312 Iteration: 5000	
loss: 0.03520692850811506 Iteration: 10000	
loss: 0.030400873314857628 Iteration: 15000	
Wall time: 5.13 s


In [180]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9854545454545455

In [181]:
weights = model.theta
weights

array([ 2.70647755, -2.55885769, -1.54498312, -1.80836148, -0.18174903])

In [182]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [183]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [184]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9854545454545455

In [185]:
accuracy.F_score()
accuracy.fScore

0.9838709677419355

In [186]:
model = LogisticRegression(lr=0.01, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "gaussian")

loss: 0.9289563626039489 Iteration: 0	
loss: 0.04862099335912129 Iteration: 5000	
loss: 0.03533020129578954 Iteration: 10000	
loss: 0.030454739708559785 Iteration: 15000	
Wall time: 5.08 s


In [187]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9854545454545455

In [188]:
weights = model.theta
weights

array([ 2.70209341, -2.55687753, -1.54411001, -1.80700861, -0.18259277])

In [189]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [190]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [191]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9854545454545455

In [192]:
accuracy.F_score()
accuracy.fScore

0.9838709677419355

In [193]:
model = LogisticRegression(lr=0.0001, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "0")

loss: 0.6928324904688002 Iteration: 0	
loss: 0.3443839969945055 Iteration: 5000	
loss: 0.2622539457986278 Iteration: 10000	
loss: 0.22195704487631504 Iteration: 15000	
Wall time: 5.04 s


In [194]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9418181818181818

In [195]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [196]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [197]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9418181818181818

In [198]:
accuracy.F_score()
accuracy.fScore

0.9327731092436974

In [211]:
model = LogisticRegression(lr=0.0001, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "uniform")

loss: 2.7126224801747054 Iteration: 0	
loss: 0.7036854148173848 Iteration: 5000	
loss: 0.355410054169544 Iteration: 10000	
loss: 0.2458675665531604 Iteration: 15000	
Wall time: 5.05 s


In [212]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9709090909090909

In [213]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [214]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [215]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9709090909090909

In [216]:
accuracy.F_score()
accuracy.fScore

0.9666666666666667

In [217]:
model = LogisticRegression(lr=0.0001, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "xavier")

loss: 2.014424372147096 Iteration: 0	
loss: 0.7186252990328104 Iteration: 5000	
loss: 0.36110994200363844 Iteration: 10000	
loss: 0.25524436625836017 Iteration: 15000	
Wall time: 5.02 s


In [218]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9672727272727273

In [219]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [220]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [221]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9672727272727273

In [222]:
accuracy.F_score()
accuracy.fScore

0.9623430962343097

In [229]:
model = LogisticRegression(lr=0.0001, num_iter=20000, verbose = True)
%time model.fit(dataClass.train_X, dataClass.train_y, weights = "gaussian")

loss: 2.67773829064788 Iteration: 0	
loss: 0.511243525067533 Iteration: 5000	
loss: 0.3201875745329449 Iteration: 10000	
loss: 0.2555407548231718 Iteration: 15000	
Wall time: 4.97 s


In [230]:
preds = model.predict(dataClass.test_X, threshold = 0.5)
# accuracy
(preds == dataClass.test_y).mean()

0.9309090909090909

In [231]:
predicted = model.predict(dataClass.test_X, threshold = 0.5)
actual = dataClass.test_y

In [232]:
class Accuracy:
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual
    def accuracy(self):
        return (self.predicted == self.actual).mean()
    def F_score(self):
        counter = Counter(zip(self.predicted, self.actual))
        self.truePositive = counter[True, 1]
        self.falsePositive = counter[True, 0]
        self.trueNegative = counter[False, 0]
        self.falseNegative = counter[False, 1]
        
        self.precision = self.truePositive / (self.truePositive + self.falsePositive)
        self.recall = self.truePositive / (self.truePositive + self.falseNegative)
        self.fScore = (2 * self.precision * self.recall) / (self.precision + self.recall)

In [233]:
accuracy = Accuracy(predicted, actual)
accuracy.accuracy()

0.9309090909090909

In [234]:
accuracy.F_score()
accuracy.fScore

0.9191489361702128